# 拉勾爬虫
## 仍存在问题：
无法持续爬取4000条内容

In [1]:
# 导包
import requests
import time
import json
from urllib.parse import quote
from lxml import etree
import csv

## 爬取列表页中的信息获取职位IP

In [2]:
# 列表页中信息的爬取
def listPageSpider(jobname):
    
    currentSite = 'Lagou'
    # jobname   = input("请输入您要搜索的关键词")
    keyword     = quote(jobname)
    # city      = quote(input("请输入您要搜索的城市"))

    # 设置开始页面的URL，即浏览器上的URL
    # https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?labelWords=&fromSearch=true&suginput=
    keywordStartURL = 'https://www.lagou.com/jobs/list_'+ keyword +'?labelWords=&fromSearch=true&suginput='

    # 设置真正的URL，即储存职位信息的URL
    keywordRealURL  = 'https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false'
    
    # 设置headers，模拟浏览器
    first_header = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36",
        'Host': 'www.lagou.com',
        'Referer': keywordStartURL,
        'Sec-Fetch-Dest': "empty",
        'Sec-Fetch-Mode': "cors",
        'Sec-Fetch-Site': "same-origin",
                        }
    headers = {
        'Host': "www.lagou.com",
        'Origin': "https://www.lagou.com",
        'Referer': keywordStartURL,
        'Sec-Fetch-Dest': "empty",
        'Sec-Fetch-Mode': "cors",
        'Sec-Fetch-Site': "same-origin",
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36",
        'X-Anit-Forge-Code': "0",
        'X-Anit-Forge-Token': "None",
        'X-Requested-With': "XMLHttpRequest"
        }
    count = 0
    for pageNumber in range(1,400):
        # 第一页：
        # payload = "first=true^&pn=1^&kd=^%^E6^%^95^%^B0^%^E6^%^8D^%^AE^%^E5^%^88^%^86^%^E6^%^9E^%^90"
        # 第二页：
        # payload = "first=false^&pn=2^&kd=^%^E6^%^95^%^B0^%^E6^%^8D^%^AE^%^E5^%^88^%^86^%^E6^%^9E^%^90^&sid=f097614de68b4d6fbba99e597d490baa"
        # 第三页：
        # payload = "first=false^&pn=3^&kd=^%^E6^%^95^%^B0^%^E6^%^8D^%^AE^%^E5^%^88^%^86^%^E6^%^9E^%^90^&sid=f097614de68b4d6fbba99e597d490baa"

        if pageNumber == 1:
            data = {
                'first': True,
                'pn': pageNumber, 
                'kd': jobname
            }
        else:
            data = {
                'first': False, 
                'pn': pageNumber, 
                'kd': jobname
            }
            
        # 每发送10个POST请求就重新获取一个cookie
        if count % 10 == 0:
            
            # 使用Session，对可见的URL发出请求，获取cookie
            requestsSession = requests.Session()
            requestsSession.get(keywordStartURL,headers = first_header)
            cookie          = requestsSession.cookies
            time.sleep(2)
            
            # 对真正储存信息的URL进行请求
            response = requestsSession.post(keywordRealURL, headers=headers, data = data,cookies = cookie) 
            count    = count+1
            time.sleep(1)
        else:
            time.sleep(2)
            
            # 对真正储存信息的URL进行请求
            response = requestsSession.post(keywordRealURL, headers=headers, data = data,cookies = cookie)
            count    = count+1
            time.sleep(1)
        yield response.json()


## 对职位详情页进行爬取

In [3]:
# 设置代理IP
def obtainProxy():
    
    # 要访问的目标页面
    # targetUrl = "http://test.abuyun.com/"
    # targetUrl = "http://proxy.abuyun.com/switch-ip"
    # targetUrl = "http://proxy.abuyun.com/current-ip"

    # 代理服务器
    proxyHost = "http-dyn.abuyun.com"
    proxyPort = "9020"

    # 代理隧道验证信息
    proxyUser = "HCP3C7O7VC10M27D"
    proxyPass = "9F1D8060D5770E94"

    proxyMeta = "http://%(user)s:%(pass)s@%(host)s:%(port)s" % {
        "host" : proxyHost,
        "port" : proxyPort,
        "user" : proxyUser,
        "pass" : proxyPass,
    }

    proxy = {
        "http"  : proxyMeta,
        "https" : proxyMeta,
    }
    return proxy


In [4]:
# 详情页中需要爬取的信息获取，方式：xpath
def detailPageInfo(detailPageURL,positionID,html,jobname):
    
    
    # 网站名称：
    dataList = ['拉勾']
    
    # 搜索关键词：
    dataList.append(jobname)
    
    # 搜索条件：
    dataList.append('https://www.lagou.com/jobs/list_'+ jobname +'?labelWords=&fromSearch=true&suginput=')
    
    # 职位ID：
    dataList.append(positionID)
    
    # 职位网址：
    dataList.append(detailPageURL)
    
    # 公司名称：
    response = html.xpath("//em[@class='fl-cn']/text()")
    dataList.append(response[0].strip())
    
    # 领域,发展阶段,投资机构,公司规模,公司网址
    response = html.xpath("//ul[@class='c_feature']//h4/text()")
    if len(response) < 5:
        response.insert(2,'NULL')
        dataList = dataList + response
    else:
        dataList = dataList + response
        
    # 职位名称
    response = html.xpath("//h1[@class='name']/text()")
    dataList.append(response[0].strip())
    
    # 标签列表
    response = html.xpath("//ul[@class='position-label clearfix']/li/text()")
    dataList.append(response)
    
    # 职位诱惑
    response = html.xpath("//dd[@class='job-advantage']//p/text()")
    dataList.append(response)
    
    # 职位描述
    response = html.xpath("//div[@class='job-detail']//p/text()")
    dataList.append(response)
    
    # 薪资范围，经验要求，学历要求，工作地点，工作性质
    # string是不可变类型,所以这里一定要重新赋值
    def process(string):
        string = string.replace('/', '') 
        return string
    response = html.xpath("//dd[@class='job_request']//span/text()")
    response = list(map(process,response))
    dataList = dataList + response
    
    # 工作地址1，2，3，查看地图
    response  = html.xpath("//body/div[@id='container']/div[@class='content_l fl']/dl[@id='job_detail']/dd[@class='job-address clearfix']/div[1]//a/text()")
    if len(response) == 3:
        response.insert(1,'NULL')
        dataList = dataList + response
    else:
        dataList = dataList + response
        
    # 发布日期
    response = html.xpath("//p[@class='publish_time']/text()")
    response = response[0].replace('\xa0','')
    dataList.append(response)
    
    return dataList


In [5]:
# 爬取详情页
def detailPageSpider(positionID,jobname,failList):
    
    # 设置详情页的URL
    detailPageURL = 'https://www.lagou.com/jobs/' + str(positionID) + '.html?show=087544dcf3cf440099dd69beffd236c3'
    
    # 设置header
    headers = {
        'Host': "www.lagou.com",
        'Referer': 'https://www.lagou.com/jobs/'+ str(positionID) + '.html?show=087544dcf3cf440099dd69beffd236c3',
        'Sec-Fetch-Dest': "empty",
        'Sec-Fetch-Mode': "cors",
        'Sec-Fetch-Site': "same-origin",
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36",
        }
    
    # 使用阿布云代理对详情页发出请求，设置参数不可重复
    result = requests.get(detailPageURL,headers = headers, proxies = obtainProxy(),allow_redirects=False).text
    html   = etree.HTML(result)
    
    # 随机获取的IP可能会不好用，出现SSL报错，所以使用try except
    try:
        htmlData = detailPageInfo(detailPageURL,positionID,html,jobname)
    
    # 将未爬取到的IP储存在failList中
    except:
        failList.append(id)
        return []
    return htmlData
    

## 整体流程：

In [6]:
# 主体流程
def main():
    
    # 定义failList为一个空列表
    failList = []
    
    # 设置搜索关键词
    jobname  = input("请输入您要搜索的关键词")
    
    # 获取ID内容
    for row in listPageSpider(jobname):
        idList = row['content']['positionResult']['result']
        for id in idList:
            try:
                time.sleep(1)
                
                # 将ID信息，搜索关键词，failList带入函数，以获取信息
                content = detailPageSpider(id['positionId'],jobname,failList)
            except:
                continue
            if content != []:
                with open('./data.csv', 'a', encoding='utf-8-sig') as f:
                    writer = csv.writer(f)
                    writer.writerow(content)
    
    # 对failList中的id进行爬取
    if failList != []:
        for id in failList:
            try:
                time.sleep(1)
                content = detailPageSpider(id,jobname,failList)
            except:
                continue
            if content != []:
                with open('./data.csv', 'a', encoding='utf-8-sig') as f:
                    writer = csv.writer(f)
                    writer.writerow(content)
                

In [7]:
main()

请输入您要搜索的关键词数据分析


ProxyError: HTTPSConnectionPool(host='www.lagou.com', port=443): Max retries exceeded with url: /jobs/positionAjax.json?needAddtionalResult=false (Caused by ProxyError('Cannot connect to proxy.', OSError("(10054, 'WSAECONNRESET')")))